# Equal-Weight S&P 500 Index Fund Application

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this Python application is to accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing I need to do is import the following open-source software libraries.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing The List of Stocks

The next thing I need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

However, I don't intend on paying for access to a index provider's API, so I'll be using a static version of the S&P 500 constituents. 

A static version of the S&P 500 constituents is available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the folder with this script so it can be accessed by other files in that directory.

Now it's time to import these stocks to the Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import the IEX Cloud API token. This is the data provider that I'll be using for this application.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. I'll be using a sandbox API token, which means that the data I'll use is randomly-generated.

You can create your own `secrets.py` file. Make sure you save the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making The First API Call

Now it's time to structure the API calls to the IEX cloud. 

I need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'SANAQD',
 'calculationPrice': 'tops',
 'open': 0,
 'openTime': None,
 'openSource': 'oiilffac',
 'close': 0,
 'closeTime': None,
 'closeSource': 'iofacfil',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 152.67,
 'latestSource': 'IEX real time price',
 'latestTime': '12:57:52 PM',
 'latestUpdate': 1672571623687,
 'latestVolume': None,
 'iexRealtimePrice': 152.53,
 'iexRealtimeSize': 5,
 'iexLastUpdated': 1671577152509,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 154.8,
 'previousVolume': 61605164,
 'change': 0.37,
 'changePercent': 0.0026,
 'volume': None,
 'iexMarketPercent': 0.014417111981444722,
 'iexVolume': 842568,
 'avgTotalVolume': 73996331,
 'iexBidP

## Parsing The API Call

The API call that I executed in the last code block contains all of the information required to build the equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. I need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding The Stocks Data to a Pandas DataFrame

The next thing I need to do is add the stock's price and market capitalization to a pandas DataFrame.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,152.67,2588798946989,N/A


## Looping Through The Tickers in The List of Stocks

Using the same logic that was outlined above, U can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True
    )

In [14]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,161.410,51139397160,N/A
1,AAL,20.700,12971225312,N/A
2,AAP,212.350,14330222878,N/A
3,AAPL,155.242,2525979091203,N/A
4,ABBV,123.230,208636619753,N/A
...,...,...,...,...
500,YUM,136.329,40735316879,N/A
501,ZBH,148.082,31938346570,N/A
502,ZBRA,578.430,31166648279,N/A
503,ZION,54.990,8917640481,N/A


## Using Batch API Calls to Improve Performance

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls I'll make in this section from 500 to 5 - huge improvement! In this section, I'll split the list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index= my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,161.410,51139397160,N/A
1,AAL,20.700,12971225312,N/A
2,AAP,212.350,14330222878,N/A
3,AAPL,155.242,2525979091203,N/A
4,ABBV,123.230,208636619753,N/A
...,...,...,...,...
500,YUM,136.329,40735316879,N/A
501,ZBH,148.082,31938346570,N/A
502,ZBRA,578.430,31166648279,N/A
503,ZION,54.990,8917640481,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, I still haven't calculated the number of shares of each stock to buy.

I'll do that next.

In [12]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:1000000


In [18]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,161.410,51139397160,12
1,AAL,20.700,12971225312,95
2,AAP,212.350,14330222878,9
3,AAPL,155.242,2525979091203,12
4,ABBV,123.230,208636619753,16
...,...,...,...,...
500,YUM,136.329,40735316879,14
501,ZBH,148.082,31938346570,13
502,ZBRA,578.430,31166648279,3
503,ZION,54.990,8917640481,36


## Formatting The Excel Output

I'll be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing the XlsxWriter Object

In [19]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats I'll Need For The `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. I'll need four main formats for the Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [20]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of The `.xlsx` File

I can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of the spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

Simplify this by putting it in 2 loops:

In [21]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving The Excel Output

Saving the Excel file is very easy:

In [22]:
writer.save()

## Saving The Excel Output

Done! This application can now succesfully take in your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund!